In [ ]:
import cv2
import numpy as np
from skimage.metrics import structural_similarity as ssim
import glob
import os
import bm3d

from Phase1.utilities.median_inpainting import *


# Define denoiser and dataset options
denoiser_options = ['BM3D']  # These are two denoisers used in the TIP paper.
dataset_options = ['classics', 'BSD68']

denoiser_choice_index = 0
dataset_choice_index = 0

denoiser_choice = denoiser_options[denoiser_choice_index]
dataset_choice = dataset_options[dataset_choice_index]

images_folder = f'./Phase1/test_sets/{dataset_choice}'

ext = ['*.jpg', '*.png', '*.bmp']
images_list = []

# Iterate over each extension
for ext_type in ext:
    # Use glob to find files matching the extension in the directory
    images_list.extend(glob.glob(os.path.join(images_folder, ext_type)))

# Get the total number of images found
N_images = len(images_list)

all_results_PSNR = np.zeros((3, N_images))
all_results_ssim = np.zeros((3, N_images))

output_log = open('./Phase1/images/output_log.txt', 'w')

for scenario_ind in range(0, 2):
    for image_ind in range(N_images):
        # Read image
        img_name = images_list[image_ind].split('/')[-1]
        X0 = cv2.imread(f'{images_folder}/{img_name}')

        # Convert to grayscale
        if len(X0.shape) > 2:
            X0 = cv2.cvtColor(X0, cv2.COLOR_BGR2GRAY)

        # Initialize parameters
        M, N = X0.shape
        n = N * M
        p = int(0.8 * n) if dataset_choice == 'classics' else int(0.5 * n)
        X0 = X0.astype(float)

        # Define parameters based on scenario
        if scenario_ind == 0:
            sig_e = 0
            maxIter = 150
            delta = 5
        elif scenario_ind == 1:
            sig_e = 10
            maxIter = 75
            delta = 0
        else:
            sig_e = 12
            maxIter = 75
            delta = 0

        # Create missing pixels
        np.random.seed(0)
        missing_pixels_ind = np.random.choice(np.random.permutation(np.arange(n)),p)
        Y_clean = X0.copy()
        Y_clean.flat[missing_pixels_ind] = 0

        # Add noise
        noise = sig_e * np.random.randn(M, N)
        Y = Y_clean + noise
        Y.flat[missing_pixels_ind] = 0

        # Run inpainting
        X_median_init = median_inpainting(Y,missing_pixels_ind)
        Y_tilde = X_median_init
        X_tilde = X_median_init
        sigma_alg = sig_e + delta

        for k in range(maxIter):
            # Estimate X_tilde
            # For denoising, use preferred method
            # Here, I'm assuming a simple median filter as a placeholder
            X_tilde = bm3d.bm3d(Y_tilde, sigma_alg, 'np')

            # Estimate Y_tilde
            Y_tilde = Y.copy()
            Y_tilde.flat[missing_pixels_ind] = X_tilde.flat[missing_pixels_ind]

            # Compute PSNR
            PSNR = 10 * np.log10(255 ** 2 / np.mean((X0 - np.clip(X_tilde, 0, 255)) ** 2))
            print(f'IDBP: finished iteration {k}, PSNR for X_tilde = {PSNR}')
            output_log.write(f'IDBP: finished iteration {k}, PSNR for X_tilde = {PSNR}\n')

        if sig_e == 0:
            # In the noiseless case, take the last Y_tilde as the estimation
            X_tilde = Y_tilde
            PSNR = 10 * np.log10(255 ** 2 / np.mean((X0 - np.clip(X_tilde, 0, 255)) ** 2))
            print(f'IDBP (noiseless case): finished iteration {k}, PSNR for X_tilde = {PSNR}')
            output_log.write(f'IDBP (noiseless case): finished iteration {k}, PSNR for X_tilde = {PSNR}\n')

        # Compute SSIM
        ssim_res = ssim(X_tilde.astype(np.uint8), X0.astype(np.uint8))

        all_results_PSNR[scenario_ind - 1, image_ind] = PSNR
        all_results_ssim[scenario_ind - 1, image_ind] = ssim_res
        print(f'scenario_ind={scenario_ind}, image_ind={image_ind}, PSNR={PSNR}, SSIM={ssim_res}')
        output_log.write(f'scenario_ind={scenario_ind}, image_ind={image_ind}, PSNR={PSNR}, SSIM={ssim_res}\n')

        # Display original, observed, and inpainted images
        # cv2.imshow('Original Image', X0.astype(np.uint8))
        # cv2.imshow('Observed Image', Y.astype(np.uint8))
        # cv2.imshow('Inpainted Image', X_tilde.astype(np.uint8))
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()

        # Save images
        folder_path = './Phase1/images/'
        cv2.imwrite(f'{folder_path}/original_image({scenario_ind},{image_ind}).png', X0.astype(np.uint8))
        cv2.imwrite(f'{folder_path}/observed_image({scenario_ind},{image_ind}).png', Y.astype(np.uint8))
        cv2.imwrite(f'{folder_path}/inpainted_image({scenario_ind},{image_ind}).png', X_tilde.astype(np.uint8))

output_log.close()


/usr/local/lib/python3.10/dist-packages/numpy/lib/nanfunctions.py:1215: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/content/Phase1/utilities/median_inpainting.py:37: RuntimeWarning: All-NaN slice encountered
  median_val = np.nanmedian(neighbors_NaN)


IDBP: finished iteration 0, PSNR for X_tilde = 24.128082497612628
IDBP: finished iteration 1, PSNR for X_tilde = 24.31163099410956
IDBP: finished iteration 2, PSNR for X_tilde = 24.47802074670333
IDBP: finished iteration 3, PSNR for X_tilde = 24.63152326358488
IDBP: finished iteration 4, PSNR for X_tilde = 24.77487981271961
IDBP: finished iteration 5, PSNR for X_tilde = 24.91042787806933
IDBP: finished iteration 6, PSNR for X_tilde = 25.039569210651518
IDBP: finished iteration 7, PSNR for X_tilde = 25.163643485675617
IDBP: finished iteration 8, PSNR for X_tilde = 25.283897353537675
IDBP: finished iteration 9, PSNR for X_tilde = 25.400774612211436
IDBP: finished iteration 10, PSNR for X_tilde = 25.51495420303049
IDBP: finished iteration 11, PSNR for X_tilde = 25.626793599712528
IDBP: finished iteration 12, PSNR for X_tilde = 25.736761899878957
IDBP: finished iteration 13, PSNR for X_tilde = 25.845041199263573
IDBP: finished iteration 14, PSNR for X_tilde = 25.951801302687898
IDBP: finis